In [29]:
from datetime import datetime, date
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass

# -------------------------------------------
# Modify the holiday class to 
# 1. Only accept Datetime objects for date.
# 2. You may need to add additional functions
# 3. You may drop the init if you are using @dataclasses
# --------------------------------------------
# @dataclass
class Holiday:
    
    def __init__ (self,name,date):
        self.name = name
        self.date = datetime.strptime(date, '%Y-%m-%d') #string to datetime conversion
    # #class attributes
    # name: str
    # date: datetime

    # String output method
    def __str__ (self):
        return self.name+': {}'.format(self.date)

    # @property
    # def name(self):
    #     print('Getting name')
    #     return self.__name

    # @property
    # def date(self):
    #     print('Getting date')
    #     return self.__date

    # @name.deleter #don't need it probably
    # def name(self):
    #     print('Deleting name')
    #     del self.__name

    # @date.deleter
    # def date(self):
    #     print('Deleting date')
    #     del self.__name

# -------------------------------------------
# The HolidayList class acts as a wrapper and container
# For the list of holidays
# Each method has pseudo-code instructions
# --------------------------------------------
class HolidayList:
    def __init__(self):
       self.innerHolidays = []
   
    def addHoliday(self):
        # Make sure holidayObj is an Holiday Object by checking the type
        # Use innerHolidays.append(holidayObj) to add holiday
        # print to the user that you added a holiday

        top_condition = True
        while top_condition:
            holiday_name = input('Put the holiday name here:')
            holiday_date_input = input('Please input the date (yyyy-mm-dd): ')
            try:
                holidayO = Holiday(holiday_name, holiday_date_input)
                self.innerHolidays.append(holidayO)
                print(f'You added {holidayO}')
                break
            except ValueError:
                print ('That is not in datetime format. Try again.')


    def findHoliday(self):
        # Find Holiday in innerHolidays
        # Return Holiday

        holiday_name = input('Put the holiday name here:')
        year = input('Enter a year (yyyy): ')
        month = input('Enter a month (mm): ')
        day = input('Enter a day (dd): ')
        date1 = f'{year}-{month}-{day}'
        holidayObj = Holiday(holiday_name, date1)
        for x in self.innerHolidays:
            if holidayObj.name == x.name and holidayObj.date == x.date:
                return x
        else:
            print('That holiday name and date is not in the list.')

                        
    def removeHoliday(self):
        # Find Holiday in innerHolidays by searching the name and date combination.
        # remove the Holiday from innerHolidays
        # inform user you deleted the holiday

        holiday = self.findHoliday()
        self.innerHolidays.remove(holiday)
        print(f' You have successfully removed {holiday}')

    def read_json(self,filelocation):
        # Read in things from json file location
        # Use addHoliday function to add holidays to inner list.
        with open(filelocation) as f:
            data = json.load(f)
            for each_pair_unit in data['holidays']:
                pair = Holiday(each_pair_unit['name'], each_pair_unit['date'])
                self.innerHolidays.append(pair)

    def save_to_json(self):
        # Write out json file to selected file.
        print('Saving Holiday List')
        saving = False
        while saving == False:
            save_input = input('Are you sure you want to save your changes? y/n').lower()
            if save_input.isalpha() and save_input == 'y' or save_input =='n':
                if save_input == 'y':
                    new_list = []
                    new_dict = {}
                    for obj in self.innerHolidays:
                        new_list.append({'name':obj.name, 'date':obj.date})
                    new_dict['holidays'] = new_list
                    json_object = json.dumps(new_dict, indent = 4, default = str)
                    with open("sample.json", "w") as outfile:
                        outfile.write(json_object)
                        saving = True
                    print('All your changes are now saved.')
                else:
                    print('The holiday list save was canceled.')
            else:
                print("That is not a y or n response, please try again.") 
        
    def scrapeHolidays(self):
        # Scrape Holidays from https://www.timeanddate.com/holidays/us/
        # Remember, 2 previous years, current year, and 2  years into the future. You can scrape multiple years by adding year to the timeanddate URL. For example https://www.timeanddate.com/holidays/us/2022
        # Check to see if name and date of holiday is in innerHolidays array
        # Add non-duplicates to innerHolidays
        # Handle any exceptions.

        def getting_HTML(url):
                request = requests.get(url)
                return request.text

        year_list = ['2020', '2021', '2022', '2023', '2024']

        for year in year_list:
            url = 'https://www.timeanddate.com/holidays/us/'
            holiday_url = url + year
            html = getting_HTML(holiday_url)
            soup = BeautifulSoup(html,'html.parser')  
            
            for tr in soup.find_all('tr'):
                title = tr.find('a')
                date = tr.find('th', class_ = 'nw')
                if title != None and date != None:
                    current = datetime.strptime(date.text + ',' + year, "%b %d,%Y")
                    date = current.strftime('%Y-%m-%d')
                    self.innerHolidays.append(Holiday(title.text, date))
                    # print(title.text)
                    # print(date)
                    # print(holiday_url)

    def numHolidays(self):
        count_numHolidays = 0
        # Return the total number of holidays in innerHolidays

        print(len(self.innerHolidays))

        # for i in self.innerHolidays:
        #     count_numHolidays += 1
        #     print('There are' + count_numHolidays +'holidays stored.')
        #     print(count_numHolidays)

    
    def filter_holidays_by_week(self, year, week_number):
        # Use a Lambda function to filter by week number and save this as holidays, use the filter on innerHolidays
        # Week number is part of the the Datetime object
        # Cast filter results as list
        # return your holidays

        holidays_by_year = list(filter(lambda x: x.date.isocalendar().year == year, self.innerHolidays))
        holidays_by_week = list(filter(lambda x: x.date.isocalendar().week == week_number, holidays_by_year))
        return holidays_by_week

    def displayHolidaysInWeek(self, holidayList):
        # Use your filter_holidays_by_week to get list of holidays within a week as a parameter
        # Output formated holidays in the week. 
        # * Remember to use the holiday __str__ method.

        for x in holidayList:
            print(x)

    # def getWeather(self, weekNum):
    #     # Convert weekNum to range between two days (my edits: it means weeks I believe)
    #     # Use Try / Except to catch problems
    #     # Query API for weather in that week range
    #     # Format weather information and return weather string.

    def viewCurrentWeek(self):
        # Use the Datetime Module to look up current week and year
        # Use your filter_holidays_by_week function to get the list of holidays 
        # for the current week/year
        # Use your displayHolidaysInWeek function to display the holidays in the week
        # Ask user if they want to get the weather
        # If yes, use your getWeather function and display results

        view_current_week = date.today().isocalendar().week
        view_current_year = date.today().isocalendar().year
        current = self.filter_holidays_by_week(view_current_year, view_current_week)
        self.displayHolidaysInWeek(current)

def main():
    # Large Pseudo Code steps
    # -------------------------------------
    # 1. Initialize HolidayList Object
    # 2. Load JSON file via HolidayList read_json function
    # 3. Scrape additional holidays using your HolidayList scrapeHolidays function.
    # 3. Create while loop for user to keep adding or working with the Calender
    # 4. Display User Menu (Print the menu)
    # 5. Take user input for their action based on Menu and check the user input for errors
    # 6. Run appropriate method from the HolidayList object depending on what the user input is
    # 7. Ask the User if they would like to Continue, if not, end the while loop, ending the program.  If they do wish to continue, keep the program going.
    
    # This is where I scrape my holiday lists for each year.

    # year = HolidayList()
     
    # for i in complete_List:
    #     this = Holiday(i['name'], i['date'])
    #     year.innerHolidays.append(this)
    #     return year.innerHolidays


    # This is where I make an instance out of my holiday list class:
    holiday_list = HolidayList()

    # This is where I read JSON data:
    holiday_list.read_json('holidays.json')

    # This is where I scrape holidays from website:
    holiday_list.scrapeHolidays()

    # This is where I call number of holidays function:
    holiday_list.numHolidays()

    # Here is my main menu with a while loop:
    
    main_menu = True
    while main_menu:

        print('Holiday Menu')
        print('==================')
        print('1. Add a Holiday')
        print('2. Remove a Holiday')
        print('3. Save Holiday List')
        print('4. View Holidays')
        print('5. Exit')

        user_input = input('Please pick a number from 1-5')
        if user_input == '1':
            holiday_list.addHoliday()
        elif user_input == '2':
            holiday_list.removeHoliday()
        elif user_input == '3':
            holiday_list.save_to_json()
        elif user_input == '4':
            user_year = input('What year would you like to view?')
            user_week = input('What week would you like to view? (if you dont enter a week, you will view the current week)')
            if user_week == '':
                holiday_list.viewCurrentWeek()
            else:
                user_not_current_week = holiday_list.filter_holidays_by_week(int(user_year),int(user_week))
                holiday_list.displayHolidaysInWeek(user_not_current_week)
        elif user_input == '5':
            main_menu = False
        else:
            print('That is not a number from 1-5. Try again.')




if __name__ == "__main__":
    main();


# Additional Hints:
# ---------------------------------------------
# You may need additional helper functions both in and out of the classes, add functions as you need to.
#
# No one function should be more then 50 lines of code, if you need more then 50 lines of code
# excluding comments, break the function into multiple functions.
#
# You can store your raw menu text, and other blocks of texts as raw text files 
# and use placeholder values with the format option.
# Example:
# In the file test.txt is "My name is {fname}, I'm {age}"
# Then you later can read the file into a string "filetxt"
# and substitute the placeholders 
# for example: filetxt.format(fname = "John", age = 36)
# This will make your code far more readable, by seperating text from code.

# Importing holiday data

 

2805
Holiday Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List
4. View Holidays
5. Exit
International Day of Epidemic Preparedness: 2021-12-27 00:00:00
Christmas Day (substitute): 2021-12-27 00:00:00
New Year Holiday: 2021-12-30 00:00:00
New Year's Day (substitute): 2021-12-31 00:00:00
New Year's Day (substitute): 2021-12-31 00:00:00
New Year's Eve: 2021-12-31 00:00:00
New Year's Eve: 2021-12-31 00:00:00
New Year's Day: 2022-01-01 00:00:00
New Year's Day: 2022-01-01 00:00:00
New Year's Day: 2022-01-01 00:00:00
Holiday Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List
4. View Holidays
5. Exit
You added xmas: 2021-12-25 00:00:00
Holiday Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List
4. View Holidays
5. Exit
International Day of Epidemic Preparedness: 2021-12-27 00:00:00
Christmas Day (substitute): 2021-12-27 00:00:00
New Year Holiday: 2021-12-30 00:00:00
New Year's Day (substitute): 2021-12-31 00:00:00
New Year's Day (substitute): 2021-12-31 00:0

KeyboardInterrupt: Interrupted by user

In [5]:
import datetime 
datetime.date.today().isocalendar().week


16